# <p style="text-align: center;"> Variational Quantum Linear Solver on Rigetti QCS </p>

<p style="text-align: center;"> Ryan LaRose, Department of Computational Mathematics, Science, and Engineering & Department of Physics and Astronomy, Michigan State University </p>

<img src="main.png" alt="Overview of VQLS">

### <p style="text-align: center;"> Abstract </p>

<p style="text-align: justify;"> The variational quantum linear solver (VQLS) [1] is an algorithm for solving linear systems on near-term quantum computers. While VQLS does not offer performance guaruntees like other quantum linear systems algorithms [2-3], the ability to run on near-term computers makes it an interesting problem-specific benchmark. In this notebook, we provide a brief tutorial of VQLS and show results of running VQLS on Rigetti QCS for example linear systems. </p>

## <p style="text-align: center;"> Notebook Setup </p>

<p style="text-align: justify;"> We use pyQuil to write the VQLS algorithm and send job submissions to Rigetti QCS. In the notebook, we use the Rigetti QVM for tutorial purposes. When running on the device, the code written in the notebook is converted to a script and executed on the QPU. </p>

In [ ]:
"""Imports."""
from itertools import product
from math import pi
import time
from typing import List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy.optimize import minimize

import pyquil
from pyquil import Program, get_qc
import pyquil.gates as gates

import vqls

# <p style="text-align: center;"> The VQLS Algorithm </p>

A linear system $A \mathbf{x} = \mathbf{b}$ is defined by a matrix $A \in \mathbb{C}^{N \times N}$ and vector $\mathbf{b} \in \mathbb{C}^{N}$. Given $A$ and $\mathbf{b}$ as input, the goal is to output $\mathbf{x} \in \mathbb{C}^{N}$, or an approximate solution $\mathbf{\bar{x}}$ where $|| \mathbf{\bar{x}} - \mathbf{x} || \le \delta$. 

### <p style="text-align: center;"> Problem Setup </p>

<p style="text-align: justify;"> How can we solve linear systems of equations with a (near-term) quantum computer? Without loss of generality, we can express the matrix $A$ is a linear combination of Paulis</p>

\begin{equation}
    A = \sum_{l = 1}^{L} a_l A_l .
\end{equation}

<p style="text-align: justify;"> The vector $|\mathbf{b}\rangle$ is assumed to have some unitary $B$ which prepares it from the ground state, i.e. $B|0\rangle = |\mathbf{b}\rangle$. We can also take $B = \sum_{m = 1}^{M} b_m B_m$ as a linear combination of Paulis. </p>

We want to prepare a state

\begin{equation}
    |\mathbf{\theta}\rangle := V(\mathbf{\theta}) |0\rangle 
\end{equation}

<p style="text-align: justify;"> which has high fidelity $| \langle \theta | \mathbf{x} \rangle |^2$ with the solution $|x\rangle$ of the linear system $A |\mathbf{x}\rangle = |\mathbf{b}\rangle $. Equivalenlty, we want a state $|\theta\rangle$ such that $A |\theta\rangle$ is close to $|\mathbf{b}\rangle$. By maximizing the fidelity $|\langle \mathbf{b} | A |\theta\rangle|^2$ where

\begin{equation}
    \langle \mathbf{b} | A |\theta\rangle = \langle 0 | B^\dagger A V(\theta) |0\rangle = \sum_{l = 1}^{L} c_l \langle 0 | B^\dagger A_l V(\theta) |0\rangle .
\end{equation}

one can obtain an approximate solution $|\theta_\text{opt}\rangle = V(\theta_\text{opt}) |0\rangle$ to the linear system. 

One way to compute the fidelity is via the Hadamard Test or extensions of the Hadamard Test [1]. Another way is described below.

### <p style="text-align: center;"> The Effective Hamiltonian Approach </p>

An interesting connection to other linear systems work [4] and variational algorithms literature [5] is to note that the above prescription is equivalent to minimizing the energy of an effective Hamiltonian

\begin{equation}
    H_{A, \mathbf{b}} \equiv H := A^\dagger ( I - |\mathbf{b}\rangle \langle \mathbf{b} | ) A
\end{equation}

<p style="text-align: justify;">
This can be seen by noting that $H$ is positive semidefinite and $H |\mathbf{x}\rangle = 0$. Thus, by minimizing $\langle \theta | H |\theta \rangle$, we can get an approximate solution to the linear system. A key advantage of this approach is the ability to compute expectation values in a hardware efficient manner. (That is, not using controlled gates required by the Hadamard test but instead using classical post-processing [5-7].) We use this <b>effective Hamiltonian approach</b> in our simulations below.
</p>

Noting that $H = A^\dagger A - A^\dagger P_{\mathbf{b}} A$ where $P_{\mathbf{b}} := |\mathbf{b}\rangle \langle \mathbf{b} |$ is the projector onto the $|\mathbf{b}\rangle$ state, we have two terms to evaluate. For the first, one can show that

\begin{equation}
    A^\dagger A = \sum_{l = 1}^{L} |a_l|^2 I + 2 \sum_{1 = l < k}^{L - 1} \text{Re} \, [a_l^* a_k ] A_l A_k .
\end{equation}

The expectation of the first summation can be computed classically, while the second requires $L (L - 1) / 2 = O(L^2)$ circuit evaluations. 

For the second term in the Hamiltonian $H$, one can see that

\begin{equation}
    A^\dagger P_{\mathbf{b}} A = \sum_{k = 1}^{L} \sum_{l = 1}^{L} a_k^* a_l A_k P_\mathbf{b} A_l .
\end{equation}

Assuming $B = \sum_{m = 1}^{M} b_m B_m$, the projector $P_\mathbf{b} = B|0\rangle \langle 0 | B^\dagger$ can be expanded as a linear combination of unitaries as well. Note that $|0\rangle\langle0| = (I + Z) / 2$. In general, there are $N M^2 L^2$ weighted Pauli operators in this term ($N = 2^n$ where $n$ is the number of qubits).  

As mentioned, the <font color="green"><b>advantage</b></font> of this approach is the short-depth circuits for cost evaluation. The <font color="red"><b>disadvantage</b></font> of this approach is that it can lead to many terms in the Pauli expansion of $H_{A, \mathbf{b}}$. An interesting observation is the "physicality" of this effective Hamiltonian -- namely, physical Hamiltonians usually have (or are assumed to have) poly($n$) terms, while this Hamiltonian has $O(N)$ terms in the worst case. We remark that simplifications which reduce the number of terms in the effective Hamiltonian $H_{A, \textbf{b}}$ are usually possible. Additionally, computing $\langle H \rangle$ by exploiting simulataneous measurements of observables can mitigate the large number of terms.

# <p style="text-align: center;"> Linear System 1 </p>

<p style="text-align: justify;"> The first linear system we consider is an example from the VQLS paper. Our motivation for this is to compare results from the paper (which used older Rigetti computers) to results we get on the new computers. The linear system is a three- to five-qubit example given by </p>

\begin{equation}
    A_1 = I + 0.2 X_1 + 0.2 X_1 Z_2 .
\end{equation}

and $\mathbf{b}_1 = H_1 H_2 H_4 H_4 |0\rangle^{\otimes n}$. Here, $I, X, Y$ and $Z$ are the usual Pauli operators

\begin{equation}
    I := \left[ \begin{matrix}
    1 & 0 \\
    0 & 1 \\
    \end{matrix} \right], \qquad 
    X := \left[ \begin{matrix}
    0 & 1 \\
    1 & 0 \\
    \end{matrix} \right], \qquad 
    Y := \left[ \begin{matrix}
    0 & -i \\
    i & 0 \\
    \end{matrix} \right], \qquad 
    Z := \left[ \begin{matrix}
    1 & 0 \\
    0 & -1 \\
    \end{matrix} \right]
\end{equation}

and subscripts indicate which qubits the Paulis act on. For example, $X_1$ means $X$ on the first qubit, i.e. $XII$ on three qubits or $X I I I I$ on five qubits.


For concreteness, we can reduce to three qubits and view the $8 \times 8$ matrix of this linear system. (Of course this works for any number of qubits, but we quickly run out of room on the screen!)

In [ ]:
"""View the matrix of a linear system."""
Acoeffs = [1., 0.2, 0.2]
Aterms = ["III", "XII", "XZI"]
print("The matrix of the linear system is:")
print(np.real(vqls.matrix(Acoeffs, Aterms)))

<font color="blue"><b>Note:</b></font> The matrix is quite sparse. This is expected for linear systems with only a few weighted Paulis, as Pauli strings are sparse matrices. 

Using the identity $H = (I + Z) / \sqrt{2}$, we can expand the $B$ unitary as 

\begin{align}
    B &= \frac{1}{2^2} (I + Z) \otimes (I + Z) \otimes I \otimes (I + Z) \otimes (I + Z) \\
      &= \frac{1}{2^2} \left[ IIIII + IIIIZ + IIIZI + IIIZZ + IZIII + IZIIZ + IZIZI + IZIZZ + \cdots \right]
\end{align}

## <p style="text-align: center;"> Three Qubit Tutorial </p>

Below we consider the three-qubit verstion of the linear system above. First, we define the linear system in terms of $A$ and $B$, then compute the effective Hamiltonian of the linear system. 

In [ ]:
"""Terms in effective Hamiltonian for first linear system."""
n = 3
Bcoeffs = [1 / 2**(n / 2)] * 2**n
paulis = [["X", "Z"]] * n
prods = list(product(*paulis))
Bterms = ["".join(p) for p in prods]
Acoeffs = [1, 0.2, 0.2]
Aterms = ["III", "XII", "XZI"]

ham = vqls.effective_hamiltonian(Acoeffs, Aterms, Bcoeffs, Bterms)
nterms = len(ham)
print("Number of terms:", nterms)
print("H = ")
for ii in range(nterms):
    print(f"{round(ham[ii][0], 3)} \t* \t{ham[ii][1]}", (lambda ii: "+" if ii < nterms - 1 else "")(ii))

To evaluate these terms with a quantum computer, we first need a quantum computer! Below we select the three-qubit lattice `Aspen-7-3Q-B`. 

In [ ]:
"""Get a quantum computer to run on."""
qcomputer = "Aspen-7-3Q-B"  # Three qubit lattice for testing optimizer
lattice = get_qc(qcomputer, as_qvm=True)  # Change to as_qvm=False to run on QC. Must have reservation.

We can visualize the connectivity of this lattice by drawing the qubit topology. 

In [ ]:
"""Visualize the qubit connectivity."""
graph = lattice.qubit_topology()
nx.draw(graph)

### <p style="text-align: center;"> Variational Ansatz </p>

The next and final ingredient we need to compute $\langle \theta | H | \theta \rangle$ is an ansatz state $|\theta\rangle$. We take this to be a product of $Y$-rotations on each qubit in the computer, obtained via the following function.

In [ ]:
"""Define an ansatz circuit."""
def yansatz(computer):
    """Returns a circuit with a product state ansatz."""
    n = len(computer.qubits())
    # Get a circuit and classical memory register
    circ = Program()
    creg = circ.declare("ro", memory_type="BIT", memory_size=n)

    # Define parameters for the ansatz
    angles = circ.declare("theta", memory_type="REAL", memory_size=n)

    # Add the ansatz
    circ += [gates.RY(angles[ii], computer.qubits()[ii]) for ii in range(n)]
    
    return circ, creg

We can visualize this ansatz below.

In [ ]:
"""Get an ansatz and visualize it."""
circ, creg = yansatz(lattice)
print(circ)

As we see, the main circuit is simply $Y$-rotations on each qubit where the parameters `theta[0], theta[1], theta[2]` are independent.

Next we define a function which inputs a particular $\theta$ value and outputs $a \langle \theta | P | \theta \rangle$ where $(a, P)$ is some weighted Pauli term in the Hamiltonian. 

In [ ]:
# TODO: Put in VQ
def expectation(angles: List[float], 
                coeff: complex, 
                pauli: str,
                ansatz: pyquil.Program,
                creg: pyquil.quilatom.MemoryReference,
                computer: pyquil.api.QuantumComputer,
                shots: int = 10000,
                verbose: bool = False) -> float:
    """Returns coeff * <\theta| paulii |\theta>.
    
    Args:
        angles: List of angles at which to evaluate coeff * <theta| pauli |theta>.
        coeff: Coefficient of Pauli term.
        pauli: Pauli string.
        ansatz: pyQuil program representing the ansatz state.
        creg: Classical register of ansatz to measure into.
        computer: QuantumComputer to execute the circuit on.
        shots: Number of times to execute the circuit (sampling statistics).
        verbose: Option for visualization/debugging.
    """
    if np.isclose(np.imag(coeff), 0.0):
        coeff = np.real(coeff)

    if set(pauli) == {"I"}:
        return coeff
    
    # Set up the circuit
    circuit = ansatz.copy()
    qubits = computer.qubits()
    measured = []
    for (q, p) in enumerate(pauli):
        if p == "X":
            circuit += [gates.H(qubits[q]), gates.MEASURE(qubits[q], creg[q])]
            measured.append(qubits[q])
        elif p == "Y":
            circuit += [gates.S(qubits[q]), gates.H(qubits[q]), gates.MEASURE(qubits[q], creg[q])]
            measured.append(qubits[q])
        elif p == "Z":
            circuit += [gates.MEASURE(qubits[q], creg[q])]
            measured.append(qubits[q])
    
    if verbose:
        print(f"Computing {coeff} x <theta|{pauli}|theta>...")
        print("\nCircuit to be executed:")
        print(circuit)
    
    # Execute the circuit
    circuit.wrap_in_numshots_loop(shots)
    executable = computer.compile(circuit)
    res = computer.run(executable, memory_map={"theta": angles})
    
    if verbose:
        print("\nResults:")
        print(f"{len(res)} total measured bit strings.")
        print(res)
    
    # Do the postprocessing
    tot = 0.0
    for vals in res:
        tot += (-1)**sum(vals)
    return coeff * tot / shots

In [ ]:
"""Unit tests for expectation."""
SHOTS = 10000
tol = 1e-1  # To be safe
assert np.isclose(expectation([0] * n, 1, "ZZZ", circ, creg,
                              lattice, shots=SHOTS), 1.0, atol=tol)
assert np.isclose(expectation([0] * n, 0.8675309, "ZZZ", circ, creg,
                              lattice, shots=SHOTS), 0.8675309, atol=tol)
assert np.isclose(expectation([0] * n, 1, "XII", circ, creg,
                              lattice, shots=SHOTS), 0.0, atol=tol)
assert np.isclose(expectation([pi / 2, 0, 0], 1, "XII", circ, creg,
                              lattice, shots=SHOTS), 1.0, atol=tol)
assert np.isclose(expectation([pi / 2, pi / 2, 0], 1, "XXI", circ, creg,
                              lattice, shots=SHOTS), 1.0, atol=tol)
assert np.isclose(expectation([pi / 2, pi / 2, pi / 2], 1, "XXX", circ, creg,
                              lattice, shots=SHOTS), 1.0, atol=tol)
assert np.isclose(expectation([pi / 2, pi / 2, 0], 1, "XXZ", circ, creg,
                              lattice, shots=SHOTS), 1.0, atol=tol)

#### TODO: Utilize simultaneous measurements

In [ ]:
# TODO: Put in vqls.py.
def energy(angles, hamiltonian, ansatz, creg, computer, shots=10000, verbose=False):
    """Returns <theta|H|theta>.
    
    Args:
        angles:
        ham:
        ansatz:
        computer:
        shots:
        verbose:
    """
    value = 0.0
    for (coeff, pauli) in hamiltonian:
        value += expectation(angles, coeff, pauli, ansatz, creg, computer, shots, verbose)
    return value

In [ ]:
"""Unit tests for energy."""
# Test 1
n = 2
test_hamiltonian = [(1.0, "II")]
test_computer = get_qc("Aspen-7-2Q-B", as_qvm=True)
test_circuit, test_creg = yansatz(test_computer)
assert np.isclose(
    energy([0] * n, test_hamiltonian, test_circuit, test_creg, test_computer, shots=10000, verbose=False),
    1.0,
    atol=1e-5
)

# Test 2
n = 2
test_hamiltonian = [(1.0, "II"), (-0.1, "IX")]
test_computer = get_qc("Aspen-7-2Q-B", as_qvm=True)
test_circuit, test_creg = yansatz(test_computer)
assert np.isclose(
    energy([0, 0], test_hamiltonian, test_circuit, test_creg, test_computer, shots=10000, verbose=False),
    1.0,
    1e-2
)
assert np.isclose(
    energy([0, pi / 2], test_hamiltonian, test_circuit, test_creg, test_computer, shots=10000, verbose=False),
    0.9,
    1e-2
)

# TODO: More unit tests!

In [ ]:
"""Cost function for linear system 1."""
def costLS1(angles):
    val = energy(angles, ham, circ, creg, lattice, shots=10000)
    print("Current angles:", angles)
    print("Current energy:", val)
    return val

In [ ]:
"""Do the optimization."""
start = time.time()
res = minimize(costLS1, x0=np.random.randn(n), method="COBYLA")
print("Runtime:", (time.time() - start) // 60, "minutes.")

In [ ]:
energy([pi / 2, pi / 2, pi / 2], ham, circ, creg, lattice)

### <p style="text-align: center;"> Compare the Quantum and Classical Solutions </p>

In [ ]:
print("Optimal angles:", res.x)

In [ ]:
"""Define function for taking (circuit, optimal angles) --> wavefunction."""
def qsolution(ansatz, opt_angles):
    """Returns the wavefunction of the ansatz at the optimal angles."""
    prog = Program()
    memory_map = {"theta": opt_angles}
    for name, arr in memory_map.items():
        for index, value in enumerate(arr):
            prog += gates.MOVE(gates.MemoryReference(name, offset=index), value)

    ansatz = prog + ansatz
    soln = pyquil.quil.percolate_declares(ansatz)
    
    wfsim = pyquil.api.WavefunctionSimulator()
    return wfsim.wavefunction(soln).amplitudes

In [ ]:
qxvec = qsolution(circ, res.x)
print(qxvec)

In [ ]:
"""Compare to classical solution."""
# Get the classical solution
Amat = matrix(Acoeffs, Aterms)
bvec = vector(Bcoeffs, Bterms)
xvec = np.linalg.solve(Amat, bvec)
xvec /= np.linalg.norm(xvec)

# Compute the fidelity
fidelity = abs(np.dot(qxvec.conj(), xvec))**2
print(r"| < xquantum | xclassical > |^2 =", round(fidelity, 2))

**SAVE THIS CELL!**

In [ ]:
"""SAVE THIS CELL!

Optimal angles for Ry product ansatz. Energy of effective Hamiltonian is near 0.001 here.
"""
LS1opt_angles = [1.60380032, 1.88421818, 1.61131353, 1.51679728, 1.44425814]

In [ ]:
def yansatzCZ(computer):
    """Returns a Ry ansatz with some entanglement."""
    # Grab the qubits
    qubits = tuple(computer.qubits())
    n = len(qubits)
    
    # Get a circuit and classical memory register
    circ = Program()
    creg = circ.declare("ro", memory_type="BIT", memory_size=n)

    # Define parameters for the ansatz
    angles = circ.declare("theta", memory_type="REAL", memory_size=n)

    # Add the ansatz
    circ += [gates.RY(angles[ii], qubits[ii]) for ii in range(n)]
    circ += [gates.CZ(qubits[ii], qubits[ii + 1]) for ii in range(n - 1)]
    
    return circ, creg

# <p style="text-align: center;"> Linear System 2 </p>

The second linear system we consider is formed from the Ising model.

\begin{equation}
    A_2 := \frac{1}{\zeta} \left(\eta I + \sum_{j=1}^{n} X_j + J \sum_{j=1}^{n-1}Z_jZ_{j+1} \right)
\end{equation}

In [ ]:
def get_Acoeffs_Aterms_LS2(n: int, zeta: float, eta: float, J: float) -> List[Tuple[complex, str]]:
    """Returns linear system 2 expressed as weighted Pauli sum."""
    if n < 0:
        raise ValueError("Number of qubits must be >= 1.")
    # Add the identity
    Acoeffs = [eta / zeta]
    Aterms = ["I" * n]
    
    # Add the X terms
    Acoeffs += [1 / zeta] * n
    xbase = "X" + "I" * (n - 1)
    for ii in range(n, 0, -1):
        Aterms.append(xbase[ii:] + xbase[:ii])
        
    # Add the ZZ terms
    Acoeffs += [J / zeta] * (n - 1)
    zzbase = "ZZ" + "I" * (n - 2)
    for ii in range(n, 1, -1):
        Aterms.append(zzbase[ii:] + zzbase[:ii])
    
    return Acoeffs, Aterms

In [ ]:
"""Unit tests for getting LS2."""
Acoeffs, Aterms = get_Acoeffs_Aterms_LS2(4, 1, 1, 0.1)
assert Acoeffs == [1.0, 1.0, 1.0, 1.0, 1.0, 0.1, 0.1, 0.1]
assert Aterms == ['IIII', 'XIII', 'IXII', 'IIXI', 'IIIX', 'ZZII', 'IZZI', 'IIZZ']

In [ ]:
"""Terms in effective Hamiltonian for second linear system."""
# Number of qubits
n = 5

# Constants
zeta = 1.0
J = 0.1
eta = 1.0

# |b> = H|0>
Bcoeffs = [1 / 2**(n / 2)] * 2**n
paulis = [["X", "Z"]] * n
prods = list(product(*paulis))
Bterms = ["".join(p) for p in prods]

# Bcoeffs = [1 / 2**(1 / 2)] * 2
# Bterms = ["IXIII", "XIXII"]

# Linear system in weighted Paulis
Acoeffs, Aterms = get_Acoeffs_Aterms_LS2(n, zeta, eta, J)

ham2 = effective_hamiltonian(Acoeffs, Aterms, Bcoeffs, Bterms)

print("Number of terms:", len(ham2))
print(*ham2, sep="\n")

In [ ]:
"""View matrix of linear system we are solving."""
print(matrix_of_hamiltonian(ham2))

In [ ]:
"""Parameters/constants."""
# Computer to run on
qcomputer = "Aspen-7-5Q-B"  # Five qubit lattice for first linear system
# qcomputer = "Aspen-7-3Q-B"  # Three qubit lattice for testing optimizer
lattice3q = get_qc(qcomputer, as_qvm=True)  # Change to as_qvm=False to run on QC. Must have reservation.

In [ ]:
"""Get an ansatz."""
circ, creg = yansatz(lattice3q)
print(circ)

In [ ]:
"""Cost function for linear system 2."""
def costLS2(angles):
    val = energy(angles, ham2, circ, lattice3q, shots=10000)
    print("Current angles:", angles)
    print("Current energy:", val)
    return val

In [ ]:
"""Do the optimization."""
start = time.time()
res = minimize(costLS2, x0=np.random.randn(n), method="COBYLA")
print("Runtime:", (time.time() - start) // 60, "minutes.")

In [ ]:
print("Optimal angles:", res.x)

In [ ]:
qxvec = qsolution(circ, res.x)
print(qxvec)

In [ ]:
"""Compare to classical solution."""
# Get the classical solution
Amat = matrix(Acoeffs, Aterms)
bvec = vector(Bcoeffs, Bterms)
xvec = np.linalg.solve(Amat, bvec)
xvec /= np.linalg.norm(xvec)

# Compute the fidelity
fidelity = abs(np.dot(qxvec.conj(), xvec))**2
print(r"| < xquantum | xclassical > |^2 =", round(fidelity, 2))

In [ ]:
xvec

## Running on a Quantum Computer

This notebook was made into a script that could be run on Rigetti's Quantum Cloud Services. In particular, we used a three qubit lattice "Aspen-4-3Q-A," the fidelity/noise characterization of which can be found online. In the cell below, we load energy vs iteration data that was computed using our VQE algorithm on this lattice. We then plot it against the simulator result to compare the difference.

In [ ]:
"""Plot the QPU energy vs iteration data obtained by running on Rigetti QCS."""
# Read in the files
qpu_energy1 = np.loadtxt("qpu-energy-iteration1.txt")
qpu_energy2 = np.loadtxt("qpu-energy-iteration2.txt")

# Do the plotting
plt.figure(figsize=(10, 5))
plt.plot(energies, "--o", linewidth=3, label="Simulator")
plt.xlabel("Iteration", fontsize=14, fontweight="bold")
plt.ylabel("Energy", fontsize=14, fontweight="bold")

plt.plot(qpu_energy1, "--o", linewidth=3, label="Raw QPU Run 1")
plt.plot(qpu_energy2, "--o", linewidth=3, label="Raw QPU Run 2")

# Put a line at the actual ground state energy (see below)
GSENERGY = 0.53232723
plt.plot(GSENERGY * np.ones_like(energies), linewidth=3, label="Analytic Energy")

plt.grid()
plt.legend();

## Error Mitigation

The QPU data above has a significant vertical shift from the simulator data, but generally the same "shape." This shift is present because of decoherence, gate application errors, measurement errors, and other noise in the system. It can be accounted for by running a set of "benchmark circuits" on the QPU prior to running the actual algorithm. These benchmark circuits are simple circuits, such as NOT and MEASURE, for which one knows the actual output. The vertical shift in this benchmark circuit can then be tested for and subtracted from the final computed energies to get more accurate results. Other methods are also possible, and this is an area of active research.

Here, we employ a similar idea, but instead of running a benchmark circuit, we just subtract off the difference of the initial energies on the QPU and on the simulator. The cell below implements this and plots the result again.

In [ ]:
"""Plot the error mitigated QPU energy vs iteration data obtained by running on Rigetti QCS."""
# Constant shift amount. In practice, this would be obtained by running a "benchmark circuit."
# Here, we just set the value based on the above curves
shift = 0.3

# Read in the files
qpu_energy1 = np.loadtxt("qpu-energy-iteration1.txt")
qpu_energy2 = np.loadtxt("qpu-energy-iteration2.txt")

# Subtract off the shift
qpu_energy1 -= shift
qpu_energy2 -= shift

# Do the plotting
plt.figure(figsize=(10, 5))
plt.plot(energies, "--o", linewidth=3, label="Simulator")
plt.xlabel("Iteration", fontsize=14, fontweight="bold")
plt.ylabel("Energy", fontsize=14, fontweight="bold")

plt.plot(qpu_energy1, "--o", linewidth=3, label="Error Mitigated QPU Run 1")
plt.plot(qpu_energy2, "--o", linewidth=3, label="Error Mitigated QPU Run 2")

# Put a line at the actual ground state energy (see below)
GSENERGY = 0.53232723
plt.plot(GSENERGY * np.ones_like(energies), linewidth=3, label="Analytic Energy")

plt.grid()
plt.legend();

## Conclusions

## Acknowledgements

Thank coauthors and Rigetti.

## References

[1] VQLS

[2] HHL

[3] QLSP via QSVE

[4] Yigit's adiabatic algorithm

[5] VQE Original paper

[6] Theory of VQE

[7] Ryan LaRose et al., [Implementing variational quantum algorithms](https://github.com/rmlarose/qcbq/blob/master/tutorials/04qaoa/qcbq_tutorial4b_implement_qaoa-INSTRUCTOR.ipynb), MSU-IBM Quantum computing bootcamp with Qiskit, 2019. 